# Cost of Living Datenanalyse - Jupyter Notebook mit API-Abfrage

In [1]:
import requests
import time
import json
import os

In [2]:
API_KEY = "" # insert key here!
headers = {
    "Authorization": f"Bearer {API_KEY}"
}

cities = [
    {"city": "vienna", "country": "austria"},
    {"city": "graz", "country": "austria"},
    {"city": "linz", "country": "austria"},
    {"city": "salzburg", "country": "austria"},

    {"city": "berlin", "country": "germany"},
    {"city": "munich", "country": "germany"},
    {"city": "hamburg", "country": "germany"},
    {"city": "dusseldorf", "country": "germany"},
    {"city": "frankfurt", "country": "germany"},
    {"city": "hannover", "country": "germany"},
    {"city": "nuremberg", "country": "germany"},
    {"city": "essen", "country": "germany"},
    {"city": "cologne", "country": "germany"},

    {"city": "bern", "country": "switzerland"},
    {"city": "zurich", "country": "switzerland"},
    {"city": "geneva", "country": "switzerland"},
    {"city": "basel", "country": "switzerland"}
]

## Define Output Path for the json file

In [3]:
output_path = "cost_of_living_raw.json"

## Send request to the API and saving

In [4]:
results = []
for loc in cities:
    params = {"city": loc["city"], "country": loc["country"]}
    response = requests.get(
        "https://zylalabs.com/api/3440/world+cost+of+living+api/3745/get+prices",
        headers=headers,
        params=params
    )
    if response.status_code == 200:
        data = response.json()
        data["city"] = loc["city"]
        data["country"] = loc["country"]
        data["timestamp"] = time.time()
        results.append(data)
    else:
        print(f"Fehler bei {loc['city']}: {response.status_code}")
    time.sleep(1)

## Sava as JSON

In [5]:
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"{len(results)} Datensätze gespeichert in '{output_path}'")

17 Datensätze gespeichert in 'cost_of_living_raw.json'
